# DQN Trading Agent

## Read database

In [1]:
#TO READ THE DATABASE
import h5py

DB_FILE = "../data/dataset_1h_1000.hdf5"
DB = h5py.File(DB_FILE, "r")

for item in DB:
    print(item)

bitcoin_usd
etherium_usd
ripple_usd


In [2]:
# Show the values of the first timestep
print(DB["bitcoin_usd"][0:2])

[[  1.51543080e+12   1.47930000e+04   1.46940000e+04   1.48600000e+04
    1.46690000e+04   7.86847895e+02]
 [  1.51542720e+12   1.44000000e+04   1.47890000e+04   1.49230000e+04
    1.43470000e+04   4.20724394e+03]]


## Define packages

In [3]:
import random
import numpy as np

## Define DQN Agent

In [4]:
from collections import deque
from keras.models import Model 
from keras.layers import Input, Dense, Flatten, Reshape
from keras.optimizers import Adam
from keras import backend as K

K.set_image_dim_ordering('tf')

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        inputs = Input(shape=(6,100))
        x = Flatten()(inputs)
        x = Dense(32, activation='relu')(x)
        x = Dense(32, activation='relu')(x)
        x = Dense(16, activation='relu')(x)
        out = Dense(self.action_size, activation='linear')(x)
        
        model = Model(inputs=inputs, outputs=out)
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        print(model.summary())
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            # The agent acts randomly
            return random.randrange(self.action_size)
        
        # Predict the reward value based on the given state
        act_values = self.model.predict(state)
        
        # Pick the action based on the predicted reward
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        # Sample minibatch from the memory
        minibatch = random.sample(self.memory, batch_size)
        
        # Extract informations from each memory
        for state, action, reward, next_state, done in minibatch:
            
            # if done, make our target reward
            target = reward
            if not done:
                # predict the future discounted reward
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
                
            # make the agent to approximately map
            # the current state to future discounted reward
            # We'll call that target_f
            target_f = self.model.predict(state)
            target_f[0][action] = target
            
            # Train the Neural Net with the state and target_f
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

Using TensorFlow backend.


## Define Trading Game

In [5]:
class TradingGame:
    def __init__(self, start_capital, state_size, database):
        self.start_capital = start_capital
        self.state_size = state_size
        
        self.database = np.asarray(database["bitcoin_usd"])
        self.capital = start_capital
        self.timestep = 0
        self.state = self.database[0:self.state_size]
        
    def reset(self):
        self.capital = self.start_capital
        self.timestep = 0
        self.state = self.database[0:self.state_size]
        
        return self.state
    
    
    ### Next state function
    def calc_next_state(self):
        self.timestep += 1
        
        if (self.state_size+self.timestep) < self.database.shape[0]:
            next_state = self.database[self.timestep:self.state_size+self.timestep]
            done = False
        else:
            next_state = self.state
            done = True
            
        return next_state, done
    
    ### Reward function
    def calc_reward(self, action, next_state):
        # Calculate price difference
        price = self.state[self.state_size-1, 2]
        last_price = next_state[self.state_size-1, 2]
        diff = last_price - price
        
        # give -1 when wrong and 1 when correct
        
        if (diff < 0 and action == 0):
            reward = 1
        elif (diff > 0 and action == 1):
            reward = 1
        else:
            reward = -1
        
        return reward
    
    ### Next step function
    def step(self, action):
        # Get next state
        next_state, done = self.calc_next_state()
        
        # Get reward
        reward = self.calc_reward(action, next_state)
        
        # check if done
        self.capital += reward
        if self.capital == 0:
            done = True
        
        # update state
        self.state = next_state
        
        return next_state, reward, done
    
    def get_score(self):
        return self.capital

## Train the agent

### Set parameters

In [6]:
# Parameter for Agent
state_size = 100 # Time frame
#state_attributes = 8
action_size = 2 # buy, sell, Hold

# Parameter for Trading
batch_size = 32
EPISODES = 5000
CAPITAL = 100
#POSITION = 0

### Training

In [ ]:
# Create agent
agent = DQNAgent(state_size, action_size)

# Initialize Environment
env = TradingGame(CAPITAL, state_size, DB)

In [8]:
#Episodes to train
for e in range(EPISODES):
    
    #Create initial state from time frame
    state = env.reset()
    state = np.reshape(state, [1, 6, state_size])

    #time frames
    for time_frame in range(state_size, 1000):
        
        #Agent takes action
        action = agent.act(state)

        #Calc reward
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, [1, 6, state_size])
        
        #Remember action
        agent.remember(state, action, reward, next_state, done)
        
        #Override state with next state
        state = next_state
        
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES, env.get_score(), agent.epsilon))
            break
       
    #Replay
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)
    # if e % 10 == 0:
    #     agent.save("./save/cartpole-dqn.h5")
        
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 6, 100)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 600)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                19232     
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 34        
Total params: 20,850
Trainable params: 20,850
Non-trainable params: 0
_________________________________________________________________
None
e

KeyboardInterrupt: 

def calc_next_state(state):
#Calc Capital (action == 0 then sell, action == 1 then buy) 5 coins * state_close_price
        #Calc position (current position -/+ differ from action)
        #Get newest state item and append capital and position
        #Remove latest entry and add new state item